In [1]:
import numpy as np

In [2]:
def xnor(a):
    res = np.zeros((2, ))
    
    if a[0] == a[2]:
        res[0] = 1
    else:
        res[0] = 0
    
    if a[1] == a[3]:
        res[1] = 1
    else:
        res[1] = 0
        
    return res

In [3]:
def make_dataset():
    data = np.zeros((16, 4))
    labels = np.zeros((16, 2))
    index = 0
    
    for a in range(2):
        for b in range(2):
            for c in range(2):
                for d in range(2):
                    data[index] = [a, b, c, d]
                    labels[index] = xnor(data[index])
                    index += 1
    
    return data, labels

In [4]:
trainset, labels = make_dataset()

In [5]:
for i in range(16):
    print(trainset[i], labels[i])

[0. 0. 0. 0.] [1. 1.]
[0. 0. 0. 1.] [1. 0.]
[0. 0. 1. 0.] [0. 1.]
[0. 0. 1. 1.] [0. 0.]
[0. 1. 0. 0.] [1. 0.]
[0. 1. 0. 1.] [1. 1.]
[0. 1. 1. 0.] [0. 0.]
[0. 1. 1. 1.] [0. 1.]
[1. 0. 0. 0.] [0. 1.]
[1. 0. 0. 1.] [0. 0.]
[1. 0. 1. 0.] [1. 1.]
[1. 0. 1. 1.] [1. 0.]
[1. 1. 0. 0.] [0. 0.]
[1. 1. 0. 1.] [0. 1.]
[1. 1. 1. 0.] [1. 0.]
[1. 1. 1. 1.] [1. 1.]


In [50]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_backward(z):
    return np.multiply(z, (1-z))

In [51]:
def init_weights():
    W1 = np.random.randn(8, 4) * 0.1
    b1 = np.random.rand(8, 1) * 0.1
    W2 = np.random.rand(2, 8) * 0.1
    b2 = np.random.rand(2, 1) * 0.1

    p = [W1, b1, W2, b2]
    
    return p

In [52]:
def forward(X, p):
    Z_ = np.matmul(p[0], X) + p[1]
    Z = sigmoid(Z_)

    A_ = np.matmul(p[2], Z) + p[3]
    A = sigmoid(A_)
    
    m = [A, Z]

    return m

In [53]:
def MSELoss(A, Y):
    loss = 1/2 * np.sum((A - Y)**2)
    return loss

In [54]:
def LogMSE(A, Y):
    loss = 1/2 * np.sum((np.log(A) - np.log(Y))**2)
    return loss

In [55]:
def backward(p, m, X, Y, lr=0.2):
    dA = - np.sum(Y - m[0], axis=0, keepdims=True)
    dA_ = dA * sigmoid_backward(m[0])

    dW2 = np.matmul(dA_, m[1].T)
    db2 = np.array(dA_, copy=True)

    dZ = np.matmul(p[2].T, dA_)
    dZ_ = dZ * sigmoid_backward(m[1])

    dW1 = np.matmul(dZ_, X.T)
    #print("dZ_: {}".format(dZ_))
    #print("X: {}".format(X))
    
    db1 = np.array(dZ_, copy=True)

    p[2] -= dW2 * lr
    p[3] -= db2 * lr
    p[0] -= dW1 * lr
    p[1] -= db1 * lr
    
    #print("dW1: {}, \ndb1: {}, \ndW2: {}, \ndb2: {}".format(dW1, db1, dW2, db2))
    
    return p

In [56]:
p = init_weights()
for epoch in range(10):
    running_loss = 0.0

    for i in range(16):
        X = trainset[i].reshape((-1, 1))
        Y = labels[i].reshape((-1, 1))
        
        m = forward(X, p)

        running_loss += MSELoss(m[0], Y)

        p = backward(p, m, X, Y, lr=0.1)

    loss = running_loss
    print("Epoch: {}, Loss: {}".format(epoch+1, loss))

Epoch: 1, Loss: 4.110367482662907
Epoch: 2, Loss: 4.089285777499239
Epoch: 3, Loss: 4.0832556680236305
Epoch: 4, Loss: 4.0815328949053225
Epoch: 5, Loss: 4.081026135276642
Epoch: 6, Loss: 4.080861099188582
Epoch: 7, Loss: 4.0807919393065175
Epoch: 8, Loss: 4.080749847945292
Epoch: 9, Loss: 4.080715738544342
Epoch: 10, Loss: 4.080684423240213


In [57]:
running_correct = 0

for i in range(16):
    X = trainset[i].reshape((-1, 1))
    Y = labels[i].reshape((-1, 1))

    m = forward(X, p)
    
    running_correct += int(np.array_equal(m[0].round(), Y))

accuracy = running_correct / 16

print("Loss: {}, Accuracy: {}".format(loss, accuracy))

Loss: 4.080684423240213, Accuracy: 0.25
